In [1]:
import observational_fear.load as load
from observational_fear.nbox_transforms import summarize_prepost_events, wilcoxon_by_cell
from observational_fear.events import get_block_starts
from observational_fear.stats import auc
from neurobox.wide_transforms import resample
from pathlib import Path


DATA_DIR = Path(r"D:\OFL\one-p experiment\data")
FIG_DIR = Path(r"C:\Users\roryl\repos\observational-fear\figs")

In [2]:
SESSION = "day2"
df = load.load_traces(DATA_DIR, session=SESSION)
df = resample(df.set_index("time"), "100ms")
df = df.reset_index().melt(id_vars=["time"])

c:\users\roryl\repos\neurobox\neurobox\wide_transforms.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["time"] = pd.to_timedelta(df.index, unit="s")


In [3]:
events = get_block_starts(SESSION, "US")
df_res = summarize_prepost_events(
    df=df,
    events=events,
    t_before=5, 
    t_after=2,
    summary_func=auc,
    cell_col="new_id"
    )

df_us = wilcoxon_by_cell(df_res, cell_col="new_id")
pct = (df_us.p < 0.05).mean() * 100
num = (df_us.p < 0.05).sum()
print(f"Num Responders: {num} ({pct}%)")

Num Responders: 153 (14.049586776859504%)


In [6]:
events = get_block_starts(SESSION, "CS")
df_res = summarize_prepost_events(
    df=df, 
    events=events,
    t_before=5,
    t_after=1,
    summary_func=auc,
    cell_col="new_id"
    )

df_cs = wilcoxon_by_cell(df_res, cell_col="new_id")
pct = (df_cs.p < 0.05).mean() * 100
num = (df_cs.p < 0.05).sum()
print(f"Num Responders: {num} ({pct}%)")

KeyError: "['time', 'value'] not in index"

In [5]:
import pandas as pd

df = pd.concat([
    df_us.reset_index().assign(block="US"),
    df_cs.reset_index().assign(block="CS"),
])
df.to_csv(DATA_DIR / "day2-block-responders.csv", index=False)

In [7]:
df

,new_id,statistic,p,diff_of_medians,block
0,0,73.0,0.014098,-5.956742,US
1,1,232.0,0.991795,-4.925334,US
2,10,159.0,0.291423,-19.372853,US
3,1000,178.0,0.442172,0.122834,US
4,1003,203.0,0.692083,1.220874,US
...,...,...,...,...,...
1084,988,231.0,0.985340,-0.246892,CS
1085,989,174.0,0.606450,-0.460521,CS
1086,994,187.0,0.680568,0.111429,CS
1087,997,85.0,0.071071,-0.230758,CS
